Use case: 1 window to "blink" 2 candidates manually with a "hot key."

In [ ]:
import os

from ipywidgets import Label, VBox
from ipyevents import Event

from astropy.coordinates import SkyCoord

from ginga.misc import Datasrc
from ginga.misc.log import get_logger
from ginga.util import wcsmod
from ginga.util.iohelper import get_fileinfo

from astrowidgets import ImageWidget

In [ ]:
# Reading ASDF in Ginga needs this to be specified early on
wcsmod.use('astropy_ape14')

In [ ]:
logger = get_logger('my viewer', log_stderr=True, log_file=None, level=30)

In [ ]:
class BlinkImageWidget(ImageWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        num_images = 10  # Max images in cache; should user be able to set this?
        self.datasrc = Datasrc.Datasrc(num_images)  # Cache

    # Need jwst package to be installed.
    def load_jwst_asdf(self, filename):
        if filename in self.datasrc:
            image = self.datasrc[filename]
        else:
            import asdf
            from ginga.AstroImage import AstroImage

            image = AstroImage(logger=self.logger)
            image.load_file(filename, data_key='data')

            self.datasrc[filename] = image

        self._viewer.set_image(image)
        
    def load_fits(self, filename):
        bnch = get_fileinfo(filename)
        
        if filename in self.datasrc:
            image = self.datasrc[filename]
            self._viewer.set_image(image)
        else:
            super().load_fits(bnch.filepath, numhdu=bnch.numhdu)
            self.datasrc[filename] = self._viewer.get_image()

In [ ]:
w1 = BlinkImageWidget(logger=logger)
w1.click_center = True

# Autozoom and autocenter only for the first time.
w1._viewer.enable_autozoom('once')
w1._viewer.enable_autocenter('once')

In [ ]:
fits_files = [
    '/redkeep/ironthrone/ssb/stginga/test_data/jw42424001001_01101_00001_nrca5_assign_wcs.fits',
    '/redkeep/ironthrone/ssb/stginga/test_data/jw42424001001_01101_00002_nrca5_assign_wcs.fits',
    '/redkeep/ironthrone/ssb/stginga/test_data/jw42424001001_01101_00003_nrca5_assign_wcs.fits']
n_files = len(fits_files)
i_file = None  # This counter will be updated by button

In [ ]:
label1 = Label('Filename')

In [ ]:
def load_image(i_file):
    # Get data off the current image before switching
    image = w1._viewer.get_image()
    data_x = None
    saved_skycoord = None
    saved_zoom_level = None
    if image is not None:
        saved_zoom_level = w1.zoom_level
        data_x, data_y = w1._viewer.get_data_xy(*w1._viewer.get_center())
        try:
            ra, dec = image.pixtoradec(data_x, data_y)
            saved_skycoord = SkyCoord(ra, dec, unit='deg')
        except Exception:
            pass

    filename = fits_files[i_file]
    label1.value = os.path.basename(filename)
    # w1.load_fits(filename)  # FITS
    w1.load_jwst_asdf(filename)  # JWST ASDF-in-FITS

    image = w1._viewer.get_image()
    
    # Center at the same place as primary viewer
    if data_x:
        center_by_xy = True
        if saved_skycoord and image.wcs.wcs:
            try:
                w1.center_on(saved_skycoord)
            except Exception:
                pass
            else:
                center_by_xy = False
        if center_by_xy:
            w1.center_on((data_x, data_y))

    if saved_zoom_level:
        w1.zoom_level = saved_zoom_level


def handle_event(event):
    global i_file

    if event['type'] != 'keydown':
        return
    
    if i_file is None:
        i_file = 0
    elif event['key'] == 'ArrowRight':
        i_file = (i_file + 1) % n_files
    elif event['key'] == 'ArrowLeft':
        i_file = (i_file - 1) % n_files
    else:
        return

    load_image(i_file)


d = Event(source=w1, watched_events=['keydown'])
d.on_dom_event(handle_event)

In [ ]:
VBox([label1, w1])

You can mouse over the image display widget and use the left/right arrow keys to blink.